In [1]:

import pandas as pd
from pandas import DataFrame
import scipy.stats as st
import numpy as np

import gensim
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import normalize 
from xgboost import XGBRegressor
import html
import re
import scipy.stats as st


In [2]:
def clean(string):
    string = html.unescape(string)
    string = string.replace("\\n", " ")
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string)
    string = re.sub(r"#", "", string)
    string = re.sub(r"\*", "", string)
    string = re.sub(r"\'m", " am", string)
    string = re.sub(r"\'s", "", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", "", string)
    string = re.sub(r"\?", " ?", string)
    return remove_stopwords(string.strip().lower())

def remove_stopwords(string):
    tokens =[token for token in string.split() if token not in stopwords.words('english')]
    
    return " ".join(tokens)



In [3]:
class Tweet(object):

    def __init__(self, id, text, emotion, score):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.score = score
def reading(path):

    lists = list()
    with open(path) as input:
        for line in input:
            line = line.strip()
            array = line.split('\t')
            lists.append(Tweet(array[0], clean(array[1]), array[2], float(array[3])))
    return lists        

Anger_train = reading('anger-ratings-0to1.train.txt')                                   #edit the path here
Anger_test  = reading('anger-ratings-0to1.test.gold.txt')

In [4]:
score_train = list()
text_train = list()
score_test = list()
text_test = list()
for tweet in Anger_train:
    text_train.append(tweet.text)
    score_train.append(float(tweet.score))

for tweet in Anger_test:
    text_test.append(tweet.text)
    score_test.append(float(tweet.score))

In [5]:
w2v1=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True, unicode_errors='ignore')

In [6]:
w2v2=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/word2vec_twitter_model.bin', binary=True, unicode_errors='ignore')

In [ ]:
f = open('/content/glove.twitter.27B.200d.txt','r')
glove1 = {}
num = 1
for line in f:
  try:
    newline = line.split()
    word = newline[0]
    embedding = [float(digit) for digit in newline[1:]]
    glove1[word] = np.array(embedding)
    num += 1
  except Exception as e:
    pass
  


In [7]:
sentiwordnet = PolynomialFeatures(3)
def sentiwordnetscore(tweet):
    
    score = np.zeros(2)
    
    for word in tweet.split():
        synsets = list(swn.senti_synsets(word))
        
        if synsets:
            score[0] += synsets[0].pos_score()
            score[1] += synsets[0].neg_score()
            
    
    return normalize(sentiwordnet.fit_transform(np.array([score]).reshape(1, -1))[0].reshape(1, -1))[0]

    

In [8]:
def w2vEmbedding(tweet, model, d):
    
    tokens = word_tokenize(tweet)
    
    vector_list = list()
    for token in tokens:
      try:
            vector_list.append(model[token])
      
      except Exception as e:
            pass

    if len(vector_list) == 0:
        vector = np.zeros(d).tolist()
    else:
        vector = sum(vector_list) / float(len(vector_list))

    return vector

def vectorize(tweets):
  frame = list()
  tmp_vector = DataFrame(list(map(lambda x: w2vEmbedding(x, w2v1, len(w2v1['word'])), tweets)))
  frame.append(tmp_vector)
  tmp_vector = DataFrame(list(map(lambda x: w2vEmbedding(x, w2v2, len(w2v2['word'])), tweets)))
  frame.append(tmp_vector)
  tmp_vector = DataFrame(list(map(lambda x: w2vEmbedding(x, glove1, len(glove1['word'])), tweets)))
  frame.append(tmp_vector)
  tmp_vector = DataFrame(list(map(lambda x: sentiwordnetscore(x), tweets)))
  frame.append(tmp_vector)

  vectors = pd.concat(frame, axis=1)

  return vectors.values.tolist()


In [9]:
text_train = vectorize(text_train)
text_test = vectorize(text_test)

In [160]:
def evaluate(pred, gold):
    if len(pred) == len(gold):

        
        # return zero correlation if predictions are constant
        if np.std(pred)==0 or np.std(gold)==0:
            return (0,0,0,0)
        
        pears_corr=st.pearsonr(pred,gold)[0]                                    
        spear_corr=st.spearmanr(pred,gold)[0]   



      
        return np.array([pears_corr,spear_corr])


model = XGBRegressor(max_depth=3, n_estimators=20000)

x_train = np.array(text_train)
score_train = np.array(score_train)
model.fit(x_train, score_train)

y_pred = model.predict(text_test)

evaluate(y_pred, score_test)

[13:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([0.58878114, 0.57041347])

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=embedding_info[1], padding="post", truncating="post", 
                                 dtype='float64')
x_test = sequence.pad_sequences(x_test, maxlen=embedding_info[1], padding="post", truncating="post",
                                dtype='float64')




conv_1 = Conv1D(300, conv_kernel, activation='relu', input_shape=x_train.shape[1:])
pool_1 = MaxPooling1D()
flat_1 = Flatten()
dense_1 = Dense(30000, activation='relu')
dense_2 = Dense(1, activation='sigmoid')
drop_1 = Dropout(rate=0.5)




def getModel():
    
    model = Sequential()

    model.add(conv_1)
    model.add(pool_1)
    

    model.add(flat_1)
    
    model.add(dense_1)
    model.add(drop_1)
    model.add(dense_2)

    model.compile(loss='mean_squared_error', optimizer="sgd")
    
    return model